<a href="https://colab.research.google.com/github/yajuna/tensorflow_pde/blob/master/Heat_equation_with_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook solves the polar heat equation in one dimension

$\rho c \frac{\partial T}{\partial t}=\frac{\partial k}{\partial r}\frac{\partial T}{\partial r}+\frac{k}{r}\frac{\partial T}{\partial r}+k\frac{\partial^2 T}{\partial r^2}+\text{source terms}$

With the assumption that $\frac{\partial k}{\partial r} = 0$, the equation simplifies to

$\rho c \frac{\partial T}{\partial t}=\frac{k}{r}\frac{\partial T}{\partial r}+k\frac{\partial^2 T}{\partial r^2}+\text{source terms}$

The initial condition is linearly interpolated from the measured core and bark temperatures at $t = 0$. Two boundary conditions are from the measured core and bark temperatures.

We aim to solve the inverse problem for determining the parameters in the original continuous PDE.

The inverse problem is formulated to be

$\frac{\partial T}{\partial t}=\lambda_1\frac{\partial T}{\partial r}+\lambda_2\frac{\partial^2 T}{\partial r^2}+\lambda_3$

and the Physics information is

$\text{residual} = \frac{\partial T}{\partial t}-\lambda_1\frac{\partial T}{\partial r}-\lambda_2\frac{\partial^2 T}{\partial r^2}-\lambda_3$


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas
from time import time

In [ ]:
# set data type and hyperparameters
# import data

DTYPE = 'float32'
tf.keras.backend.set_floatx(DTYPE)

EPOCH = 5000

##### measured tree temperature for initial and boundary conditions. Need fixing
# colnames_tree_temp = ['datetime', 's45_1', 'e9_1', 'n135_1','e45_2', 'n9_2', 'w135_2', 'n45_3', 'w9_3','s135_3', 'w_ext_35']
# url1 = "https://raw.githubusercontent.com/yajuna/linearRegression/master/Tree_Temp_Values_OCT21_to_OCT28_2022.xlsx"
# dataTemp = pandas.read_excel(url1,names=colnames_tree_temp)

# train_tree_temp_index = 416 - 2

# train_interp_temp_size = train_tree_temp_index

# ### Initial and boundary conditions
# # core temp is west, at 13.5cm, at 2m high
# test_coreTemp = np.array(dataTemp.s135_3[train_tree_temp_index: test_tree_temp_index])+ 273.15
# # West, at 9cm, at 3m high
# test_midTemp1 = np.array(dataTemp.w9_3[train_tree_temp_index: test_tree_temp_index])+ 273.15
# # North, at 4.5cm, at 3m high
# test_midTemp2 = np.array(dataTemp.n45_3[train_tree_temp_index: test_tree_temp_index])+ 273.15
# # bark temp is West, at bark, at 3.5m high
# test_barkTemp = np.array(dataTemp.w_ext_35[train_tree_temp_index: test_tree_temp_index])+ 273.15

# # linear interpolate the measured temperature
# test_coreTemp = np. interp(time, np.linspace(0,24,test_interp_temp_size),test_coreTemp)
# test_midTemp1 = np. interp(time, np.linspace(0,24,test_interp_temp_size),test_midTemp1)
# test_midTemp2 = np. interp(time, np.linspace(0,24,test_interp_temp_size),test_midTemp2)
# test_barkTemp = np. interp(time, np.linspace(0,24,test_interp_temp_size),test_barkTemp)


# initTemp = np.array([test_coreTemp[0], test_midTemp1[0], test_midTemp2[0], test_barkTemp[0]])
# init_temp = np.interp(np.linspace(0,radius,n_x), np.linspace(0,radius,initTemp.size),initTemp)

Linearly interpolate initial and boundary data according to the generated time and space collocation points.

In [ ]:
# Set number of data points
N_0 = 500 # number of points in space
N_b = 500 # number of points in time
N_r = 100000

# Set boundary
tmin = 0.
tmax = 24.
xmin = 0.
xmax = 0.135

# Lower bounds in time and space
lb = tf.constant([tmin, xmin], dtype=DTYPE)
# Upper bounds in time and space
ub = tf.constant([tmax, xmax], dtype=DTYPE)

# Set random seed for reproducible results
tf.random.set_seed(0)

# Draw uniform sample points for initial boundary data; need N_0 == N_b for concat
t_0 = tf.ones((N_0,1), dtype=DTYPE)*lb[0]
x_0 = tf.random.uniform((N_0,1), lb[1], ub[1], dtype=DTYPE)
X_0 = tf.concat([t_0, x_0], axis=1)

# initial condition
u_0 =

# Boundary data- we compare left and right boundary at give time steps.
t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype=DTYPE)
x_lb = tf.ones((N_b,1), dtype=DTYPE)*lb[1]
x_ub = tf.ones((N_b,1), dtype=DTYPE)*ub[1]
X_lb = tf.concat([t_b, x_lb], axis=1)
X_ub = tf.concat([t_b, x_ub], axis=1)

# Boundary condition are evaluated in the loss function
# u(t_b, x_lb) = u(t_b, x_ub)

# Draw uniformly sampled collocation points
t_r = tf.random.uniform((N_r,1), lb[0], ub[0], dtype=DTYPE)
x_r = tf.random.uniform((N_r,1), lb[1], ub[1], dtype=DTYPE)
X_r = tf.concat([t_r, x_r], axis=1)

# # Collect boundary and inital data in lists<----- not needed in this code; no u_b
# X_data = [X_0, X_b]
# u_data = [u_0, u_b]